Trabalho sobre análise exploratória de dados utilizando Python, Linguagem SQL e Banco de Dados SQLite. 
Os dados utilizados nesse projeto são dados reais, disponíveis a partir do IMDB.

Realizando a instalação e carregamento dos pacotes a serem utilizados

In [1]:
# Instalando o pacote
!pip install -q imdb-sqlite

In [2]:
# instalando o pacote pycountry (nomes dos paises, etc...)
!pip install -q pycountry

In [3]:
# Importando pacotes para realização das análises numéricas

import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style = "whitegrid")

Carregamento dos Dados utilizados

In [4]:
%%time
!imdb-sqlite

2022-03-15 21:38:40,201 GET https://datasets.imdbws.com/name.basics.tsv.gz -> downloads/name.basics.tsv.gz
2022-03-15 21:39:14,898 GET https://datasets.imdbws.com/title.basics.tsv.gz -> downloads/title.basics.tsv.gz
2022-03-15 21:39:34,836 GET https://datasets.imdbws.com/title.akas.tsv.gz -> downloads/title.akas.tsv.gz
2022-03-15 21:40:09,033 GET https://datasets.imdbws.com/title.principals.tsv.gz -> downloads/title.principals.tsv.gz
2022-03-15 21:41:02,233 GET https://datasets.imdbws.com/title.episode.tsv.gz -> downloads/title.episode.tsv.gz
2022-03-15 21:41:10,918 GET https://datasets.imdbws.com/title.ratings.tsv.gz -> downloads/title.ratings.tsv.gz
2022-03-15 21:41:11,356 Populating database: imdb.db
2022-03-15 21:41:11,356 Applying schema
2022-03-15 21:41:11,357 Importing file: downloads/name.basics.tsv.gz
2022-03-15 21:41:11,357 Reading number of rows ...
2022-03-15 21:41:14,872 Inserting rows into table: people
100%|████████████████████████| 11484457/11484457 [01:02<00:00, 184084

In [5]:
# abrindo conexão ao banco de dados
conn = sqlite3.connect("imdb.db")

In [6]:
# Realizando a extração das listas de tabelas
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [8]:
# type(tabelas)
tabelas.head()

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes


In [10]:
tabelas = tabelas["Table_Name"].values.tolist()

In [12]:
# Verificando o esquema de cada tabela
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela:", tabela)
    display(resultado)
    print("-"*100)
    print("\n")

Esquema da tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: akas


...

----------------------------------------------------------------------------------------------------


Esquema da tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


